In [ ]:
!pip install anomalib
!pip install lightning

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.8/851.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.5/240.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.9/827.9 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.5/760.5 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 42.4 MB/s eta 0:00:00
  Created wheel for freia: filename=FrEIA-0.2-py3-none-any.whl size=42763 sha256=a411141169e3303aeb4de725d36609eb26d271d4ce1023c1e6a76cf7513c2f94
  Stored in di

In [ ]:
import numpy as np
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from matplotlib import pyplot as plt
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
from anomalib.data import PredictDataset, MVTec, Visa
from anomalib.engine import Engine
from anomalib.models import EfficientAd
from anomalib.models.image.efficient_ad.torch_model import EfficientAdModelSize
from anomalib.utils.post_processing import superimpose_anomaly_map
from anomalib import TaskType
from anomalib.deploy.inferencers import TorchInferencer, OpenVINOInferencer
from anomalib.metrics import AUROC, AUPRO
from anomalib.loggers import AnomalibTensorBoardLogger
import timeit
import torch
from collections import defaultdict
import os
from anomalib.data import Folder
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import torch
from pathlib import Path
from PIL import Image



In [ ]:
from lightning.pytorch import Trainer, seed_everything
seed_everything(42, workers=True)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


42

In [ ]:
import os

!mkdir -p /content/procitex # Ensure target directory exists
# Check if procitex.zip exists and is valid before unzipping
if not os.path.exists('procitex.zip'):
    print("Error: procitex.zip not found. Please ensure the zip file is present in the /content directory.")
else:
    # Attempt to unzip to /content, expecting procitex/train etc. inside the zip.
    # If the zip contains a top-level 'procitex' folder, this will result in /content/procitex
    !unzip -o procitex.zip -d /content/
    print("Unzip command executed. Please verify contents of /content/procitex")

Archive:  procitex.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of procitex.zip or
        procitex.zip.zip, and cannot find procitex.zip.ZIP, period.


In [ ]:
root_folder = "procitex/procitex"

# Percorre todas as pastas e subpastas
for dirpath, dirnames, filenames in os.walk(root_folder):
    for filename in filenames:
        if filename.endswith(".JPEG"):  # verifica arquivos .JPEG
            old_path = os.path.join(dirpath, filename)
            new_filename = filename.replace(".JPEG", ".jpeg")
            new_path = os.path.join(dirpath, new_filename)
            os.rename(old_path, new_path)  # renomeia inplace

print("Conversão concluída em todas as pastas!")

Conversão concluída em todas as pastas!


In [ ]:
root_path = os.path.abspath("/content/procitex/procitex")

data_module = Folder(
    name="Database/",
    root=root_path,
    normal_dir="train",
    abnormal_dir="test",
    seed=42,
    train_batch_size=1,

)

In [ ]:
data_module.setup()

FileNotFoundError: Path does not exist: /content/procitex/train

In [ ]:
TenLogger = AnomalibTensorBoardLogger("logs/efficient_ad/tb_logs", name="BaseModel_w_padding_procitex", log_graph=True)


callbacks = [
    ModelCheckpoint(
    monitor="train_loss_epoch",
    mode="min",
        filename="BaseModel_w_padding.ckpt"
    ),
    EarlyStopping(monitor="train_loss", mode="min", patience=10),
]

# Setup the datamodule
data_module.setup()

model_w_padding = EfficientAd(padding=True, pad_maps=False)
engine = Engine( callbacks=callbacks, logger=TenLogger, max_epochs=50)

# Train the model
engine.fit(model_w_padding, data_module)


INFO:lightning_fabric.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:lightning_fabric.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO: 
  | Name           | Type             | Params | Mode 
------------------------------------------------------------
0 | pre_processor  | PreProcessor     | 0      | train
1 | post_processor | PostProcessor    | 0      | train
2 | evaluator    

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
Calculate teacher channel mean & std: 100%|██████████| 64/64 [00:04<00:00, 13.15it/s]


Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 2/2 [00:04<00:00,  2.44s/it]


...

Calculate Validation Dataset Quantiles: 100%|██████████| 2/2 [00:06<00:00,  3.00s/it]


Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 2/2 [00:06<00:00,  3.06s/it]


Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 2/2 [00:06<00:00,  3.14s/it]
INFO:lightning_fabric.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


In [ ]:
import time
import torch
from torchvision import transforms
from PIL import Image
from pathlib import Path

# Caminho do checkpoint salvo após o treino
ckpt_path = "results/EfficientAd/Database/latest/weights/lightning/model.ckpt"

# Carrega o modelo
model = EfficientAd.load_from_checkpoint(ckpt_path)
test_root = Path("/content/procitex/procitex/test")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Threshold do modelo
try:
    threshold = model.threshold.value.item()
except Exception:
    threshold = 0.7
print(f"Threshold usado: {threshold:.4f}")

# Função para encontrar imagens de várias extensões
def find_images(folder):
    exts = ["*.png", "*.jpg", "*.jpeg"]
    images = []
    for ext in exts:
        images.extend(folder.glob(ext))
    return images

# ==========================
# Avaliar pasta "good"
# ==========================
good_folder = test_root / "good"
if good_folder.exists():
    good_images = find_images(good_folder)
    correct_good = 0
    total_good = len(good_images)

    total_time = 0.0  # 🔹 acumulador de tempo

    for img_path in good_images:
        image = Image.open(img_path).convert("RGB")
        image_tensor = preprocess(image).unsqueeze(0).to(device)

        torch.cuda.synchronize() if torch.cuda.is_available() else None
        start_time = time.time()

        with torch.no_grad():
            pred = model(image_tensor)
            score = pred.pred_score.cpu().item()

        torch.cuda.synchronize() if torch.cuda.is_available() else None
        total_time += time.time() - start_time

        pred_binary = 1 if score > threshold else 0
        if pred_binary == 0:
            correct_good += 1

    avg_time = total_time / total_good if total_good > 0 else 0
    fps = 1 / avg_time if avg_time > 0 else 0
    percent_correct_good = 100 * correct_good / total_good if total_good > 0 else 0

    print(f"GOOD: {percent_correct_good:.2f}% das imagens boas classificadas corretamente ({correct_good}/{total_good})")
    print(f" Tempo médio por imagem (GOOD): {avg_time*1000:.2f} ms ({fps:.2f} FPS)")
else:
    print(" Pasta 'good' não encontrada!")

# ==========================
# 🔹 Avaliar pastas defeituosas
# ==========================
defective_folders = [f for f in test_root.iterdir() if f.is_dir() and f.name != "good"]

for folder in defective_folders:
    images = find_images(folder)
    if not images:
        print(f" Nenhuma imagem encontrada na pasta {folder.name}")
        continue

    correct_defective = 0
    total_defective = 0
    total_time = 0.0

    for img_path in images:
        image = Image.open(img_path).convert("RGB")
        image_tensor = preprocess(image).unsqueeze(0).to(device)

        torch.cuda.synchronize() if torch.cuda.is_available() else None
        start_time = time.time()

        with torch.no_grad():
            pred = model(image_tensor)
            score = pred.pred_score.cpu().item()

        torch.cuda.synchronize() if torch.cuda.is_available() else None
        total_time += time.time() - start_time

        pred_binary = 1 if score > threshold else 0
        total_defective += 1
        if pred_binary == 1:
            correct_defective += 1

    avg_time = total_time / total_defective if total_defective > 0 else 0
    fps = 1 / avg_time if avg_time > 0 else 0
    percent_correct = 100 * correct_defective / total_defective

    print(f" {folder.name}: {percent_correct:.2f}% das imagens defeituosas detectadas ({correct_defective}/{total_defective})")
    print(f"Tempo médio por imagem ({folder.name}): {avg_time*1000:.2f} ms ({fps:.2f} FPS)")


🔹 Threshold usado: 0.7000
GOOD: 45.00% das imagens boas classificadas corretamente (9/20)
 Tempo médio por imagem (GOOD): 59.49 ms (16.81 FPS)
 cadarco: 67.86% das imagens defeituosas detectadas (19/28)
Tempo médio por imagem (cadarco): 56.80 ms (17.61 FPS)
 costura: 74.19% das imagens defeituosas detectadas (23/31)
Tempo médio por imagem (costura): 56.60 ms (17.67 FPS)
 Nenhum: 100.00% das imagens defeituosas detectadas (3/3)
Tempo médio por imagem (Nenhum): 57.26 ms (17.46 FPS)
 sujidade: 71.43% das imagens defeituosas detectadas (5/7)
Tempo médio por imagem (sujidade): 57.03 ms (17.54 FPS)
 corte: 87.50% das imagens defeituosas detectadas (21/24)
Tempo médio por imagem (corte): 56.71 ms (17.63 FPS)
